In the second part of my data analysis project i want to save all of the 24h data as an netcdf files which are more nicer to use in this case

In [4]:
import pandas as pd
import numpy as np
import xarray as xr
import glob
import os
import re

We define a function for reading the data which is based on the first notebook

In [6]:
def read_data(name = "PM10_24g"):
    root_folder = "./"
    pattern = os.path.join(root_folder, "[0-9][0-9][0-9][0-9]", f"[0-9][0-9][0-9][0-9]_{name}.xlsx")
    plik_lista = glob.glob(pattern)

    df_list = []

    for path in plik_lista:
        try:
            print(f"Wczytywanie: {path}")

            # year extraction
            fit = re.search(r'(\d{4})', path)
            if fit:
                rok = int(fit.group(1))
            else:
                print(f"Nie można rozpoznać roku z: {path}")
                continue

            # Determine the number of rows to skip based on the year
            if rok <= 2015:
                skiprows = 0  # Nagłówek to 1. wiersz
            else:
                skiprows = 1  # Nagłówek to 2. wiersz

            df = pd.read_excel(path, skiprows=skiprows, header=0, decimal=",")

            if rok <= 2015:
                df = df.drop(index=[0,1], errors="ignore")  # Usuwa 2. i 3. wiersz
            else:
                df = df.drop(index=[0,1,2,3], errors="ignore")  # Usuwa 2–5. wiersz

            df.rename(columns={df.columns[0]: "Data"}, inplace=True)
            df["Data"] = pd.to_datetime(df["Data"], errors='coerce')
            df.set_index("Data", inplace=True)

            df_list.append(df)

        except Exception as e:
            print(f"Błąd w pliku {path}: {e}")

    df_all = pd.concat(df_list, axis=0).sort_index()

    # the conversion to numeric: this time we use the default pandas method
    #def clean_column(series):
    #    s = series.astype(str).str.strip()
    #    s = s.str.replace(',', '.', regex=False)
    #    s = s.replace(r'[^0-9\.\-]', '', regex=True)
    #    return pd.to_numeric(s, errors='coerce')
    # 2. Zastosuj do całego DataFrame
    #df_all = df_all.apply(clean_column)
    df_all = df_all.apply(pd.to_numeric, errors='coerce')

    df_meta = pd.read_excel("./meta.xlsx", decimal=",")
    df_meta.set_index("Nr", inplace = True, drop = True)
    df_meta.columns = df_meta.columns.str.strip().str.lower().str.replace(r'\s+', '_', regex=True)
    df_meta = df_meta.rename(columns={'stary_kod_stacji_(o_ile_inny_od_aktualnego)': 'stary_kod'})
    df_meta = df_meta.rename(columns={
    'wgs84_φ_n': 'lat',
    'wgs84_λ_e': 'lon'
    })

    alias_map = {}
    # we perform some alias mapping to unify station names
    for _, row in df_meta.iterrows():
        kod = row['kod_stacji']

        # add the main station code as an alias of itself
        if pd.notna(kod):
            alias_map[kod] = kod

        # if the station has an old code, we add it as an alias
        if pd.notna(row['stary_kod']):
            for alias in str(row['stary_kod']).split(','):
                alias = alias.strip()
                if alias:
                    alias_map[alias] = kod

    df_renamed = df_all.copy()

    # New columns are assigned according to alias_map
    new_columns = {
        col: alias_map.get(col, col) for col in df_renamed.columns
    }

    df_renamed = df_renamed.rename(columns=new_columns)
    print(df_renamed)

    # Aggregation: e.g., mean, but you can also use .first(), .max(), etc.
    df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)
    print(df_merged)

    df_avg = df_merged.mean().reset_index()
    df_avg.columns = ['kod_stacji', 'srednia']

    df_avg = df_avg.merge(df_meta, on='kod_stacji', how='left')
    missing_coords = df_avg[df_avg[['lat', 'lon']].isna().any(axis=1)]
    print("⚠️ Stacje bez współrzędnych:")
    print(missing_coords[['kod_stacji']])
    df_avg = df_avg.dropna(subset=['lat', 'lon'])

    return df_merged, df_avg

Now define function to adding the maps as html files

In [7]:
import folium
from folium import plugins
from branca.colormap import linear as cm_linear

def get_map2(df_avg, name="mapa_stacje", with_heatmap=True):
    name = name + ".html"

    # Kolory wg średniej
    vmin = df_avg['srednia'].min()
    vmax = df_avg['srednia'].max()
    colormap = cm_linear.YlOrRd_09.scale(vmin, vmax)

    # Mapa
    m = folium.Map(location=[52.0, 19.0], zoom_start=6, control_scale=True)

    # Grupy typów stacji
    groups = {}
    for typ in df_avg['typ_stacji'].dropna().unique():
        groups[typ] = folium.FeatureGroup(name=f"Typ: {typ}", show=True)
        groups[typ].add_to(m)

    # Dodanie punktów
    for _, row in df_avg.iterrows():
        color = colormap(row['srednia'])
        popup_text = f"""
        <b>Kod stacji:</b> {row['kod_stacji']}<br>
        <b>Nazwa stacji:</b> {row['nazwa_stacji']}<br>
        <b>Data uruchomienia:</b> {row['data_uruchomienia']}<br>
        <b>Data zamknięcia:</b> {row['data_zamknięcia']}<br>
        <b>Typ stacji:</b> {row['typ_stacji']}<br>
        <b>Typ obszaru:</b> {row['typ_obszaru']}<br>
        <b>Rodzaj stacji:</b> {row['rodzaj_stacji']}<br>
        <b>Województwo:</b> {row['województwo']}<br>
        <b>Miejscowość:</b> {row['miejscowość']}<br>
        <b>Adres:</b> {row['adres']}<br>
        <b>Średnia:</b> {row['srednia']:.2f}
        """

        marker = folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=10,
            color=color,
            fill=True,
            fill_opacity=0.8,
            popup=folium.Popup(popup_text, max_width=300),
            tooltip=f"{row['nazwa_stacji']} ({row['kod_stacji']})<br>Średnia: {row['srednia']:.2f}",
        )

        group = groups.get(row['typ_stacji'])
        if group:
            marker.add_to(group)
        else:
            marker.add_to(m)

    # Legenda
    colormap.caption = 'Średnie zanieczyszczenie'
    colormap.add_to(m)

    # Kontrola warstw
    folium.LayerControl(collapsed=False).add_to(m)

    # Zapis
    m.save(name)
    print(f"✔️ Zapisano mapę: {name}")

We use all of the available names

In [8]:
names = [ 
    "Cd(PM10)_24g", "Pb(PM10)_24g", "As(PM10)_24g", "Ni(PM10)_24g",
    "BaA(PM10)_24g", "BaP(PM10)_24g", "BbF(PM10)_24g", "BjF(PM10)_24g",
    "BkF(PM10)_24g", "DBahA(PM10)_24g", "IP(PM10)_24g", "Jony_PM25_24g",
    "NO2_24g", "PM10_24g", "SO2_24g", "PM25_24g"
]

for name in names:
    print(f"\n📊 Przetwarzam: {name}")
    
    df_merged, df_avg = read_data(name)
    
    # Safe to netcdf
    ds = df_merged.to_xarray()
    ds.to_netcdf(f"{name}_merged.nc")

    # Generating a map
    map_name = name.replace("_24g", "")
    get_map2(df_avg, name=map_name)


📊 Przetwarzam: Cd(PM10)_24g
Wczytywanie: .\2001\2001_Cd(PM10)_24g.xlsx
Wczytywanie: .\2002\2002_Cd(PM10)_24g.xlsx
Wczytywanie: .\2003\2003_Cd(PM10)_24g.xlsx
Wczytywanie: .\2004\2004_Cd(PM10)_24g.xlsx
Wczytywanie: .\2005\2005_Cd(PM10)_24g.xlsx
Wczytywanie: .\2006\2006_Cd(PM10)_24g.xlsx
Wczytywanie: .\2007\2007_Cd(PM10)_24g.xlsx
Wczytywanie: .\2008\2008_Cd(PM10)_24g.xlsx
Wczytywanie: .\2009\2009_Cd(PM10)_24g.xlsx
Wczytywanie: .\2010\2010_Cd(PM10)_24g.xlsx
Wczytywanie: .\2011\2011_Cd(PM10)_24g.xlsx
Wczytywanie: .\2012\2012_Cd(PM10)_24g.xlsx
Wczytywanie: .\2013\2013_Cd(PM10)_24g.xlsx
Wczytywanie: .\2014\2014_Cd(PM10)_24g.xlsx
Wczytywanie: .\2015\2015_Cd(PM10)_24g.xlsx
Wczytywanie: .\2016\2016_Cd(PM10)_24g.xlsx
Wczytywanie: .\2017\2017_Cd(PM10)_24g.xlsx
Wczytywanie: .\2018\2018_Cd(PM10)_24g.xlsx
Wczytywanie: .\2019\2019_Cd(PM10)_24g.xlsx
Wczytywanie: .\2020\2020_Cd(PM10)_24g.xlsx
Wczytywanie: .\2021\2021_Cd(PM10)_24g.xlsx
Wczytywanie: .\2022\2022_Cd(PM10)_24g.xlsx
Wczytywanie: .\2023\2023_

C:\Users\szymo\AppData\Local\Temp\ipykernel_948\1181867703.py:90: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)


✔️ Zapisano mapę: Cd(PM10).html

📊 Przetwarzam: Pb(PM10)_24g
Wczytywanie: .\2002\2002_Pb(PM10)_24g.xlsx
Wczytywanie: .\2003\2003_Pb(PM10)_24g.xlsx
Wczytywanie: .\2004\2004_Pb(PM10)_24g.xlsx
Wczytywanie: .\2005\2005_Pb(PM10)_24g.xlsx
Wczytywanie: .\2006\2006_Pb(PM10)_24g.xlsx
Wczytywanie: .\2007\2007_Pb(PM10)_24g.xlsx
Wczytywanie: .\2008\2008_Pb(PM10)_24g.xlsx
Wczytywanie: .\2009\2009_Pb(PM10)_24g.xlsx
Wczytywanie: .\2010\2010_Pb(PM10)_24g.xlsx
Wczytywanie: .\2011\2011_Pb(PM10)_24g.xlsx
Wczytywanie: .\2012\2012_Pb(PM10)_24g.xlsx
Wczytywanie: .\2013\2013_Pb(PM10)_24g.xlsx
Wczytywanie: .\2014\2014_Pb(PM10)_24g.xlsx
Wczytywanie: .\2015\2015_Pb(PM10)_24g.xlsx
Wczytywanie: .\2016\2016_Pb(PM10)_24g.xlsx
Wczytywanie: .\2017\2017_Pb(PM10)_24g.xlsx
Wczytywanie: .\2018\2018_Pb(PM10)_24g.xlsx
Wczytywanie: .\2019\2019_Pb(PM10)_24g.xlsx
Wczytywanie: .\2020\2020_Pb(PM10)_24g.xlsx
Wczytywanie: .\2021\2021_Pb(PM10)_24g.xlsx
Wczytywanie: .\2022\2022_Pb(PM10)_24g.xlsx
Wczytywanie: .\2023\2023_Pb(PM10)_24

C:\Users\szymo\AppData\Local\Temp\ipykernel_948\1181867703.py:90: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)


✔️ Zapisano mapę: Pb(PM10).html

📊 Przetwarzam: As(PM10)_24g
Wczytywanie: .\2002\2002_As(PM10)_24g.xlsx
Wczytywanie: .\2003\2003_As(PM10)_24g.xlsx
Wczytywanie: .\2004\2004_As(PM10)_24g.xlsx
Wczytywanie: .\2005\2005_As(PM10)_24g.xlsx
Wczytywanie: .\2006\2006_As(PM10)_24g.xlsx
Wczytywanie: .\2007\2007_As(PM10)_24g.xlsx
Wczytywanie: .\2008\2008_As(PM10)_24g.xlsx
Wczytywanie: .\2009\2009_As(PM10)_24g.xlsx
Wczytywanie: .\2010\2010_As(PM10)_24g.xlsx
Wczytywanie: .\2011\2011_As(PM10)_24g.xlsx
Wczytywanie: .\2012\2012_As(PM10)_24g.xlsx
Wczytywanie: .\2013\2013_As(PM10)_24g.xlsx
Wczytywanie: .\2014\2014_As(PM10)_24g.xlsx
Wczytywanie: .\2015\2015_As(PM10)_24g.xlsx
Wczytywanie: .\2016\2016_As(PM10)_24g.xlsx
Wczytywanie: .\2017\2017_As(PM10)_24g.xlsx
Wczytywanie: .\2018\2018_As(PM10)_24g.xlsx
Wczytywanie: .\2019\2019_As(PM10)_24g.xlsx
Wczytywanie: .\2020\2020_As(PM10)_24g.xlsx
Wczytywanie: .\2021\2021_As(PM10)_24g.xlsx
Wczytywanie: .\2022\2022_As(PM10)_24g.xlsx
Wczytywanie: .\2023\2023_As(PM10)_24

...

✔️ Zapisano mapę: PM10.html

📊 Przetwarzam: SO2_24g
Wczytywanie: .\2000\2000_SO2_24g.xlsx
Wczytywanie: .\2001\2001_SO2_24g.xlsx
Wczytywanie: .\2002\2002_SO2_24g.xlsx
Wczytywanie: .\2003\2003_SO2_24g.xlsx
Wczytywanie: .\2004\2004_SO2_24g.xlsx
Wczytywanie: .\2005\2005_SO2_24g.xlsx
Wczytywanie: .\2006\2006_SO2_24g.xlsx
Wczytywanie: .\2007\2007_SO2_24g.xlsx
Wczytywanie: .\2008\2008_SO2_24g.xlsx
Wczytywanie: .\2009\2009_SO2_24g.xlsx
Wczytywanie: .\2010\2010_SO2_24g.xlsx
Wczytywanie: .\2011\2011_SO2_24g.xlsx
Wczytywanie: .\2012\2012_SO2_24g.xlsx
Wczytywanie: .\2013\2013_SO2_24g.xlsx
Wczytywanie: .\2014\2014_SO2_24g.xlsx
Wczytywanie: .\2015\2015_SO2_24g.xlsx
Wczytywanie: .\2016\2016_SO2_24g.xlsx
Wczytywanie: .\2017\2017_SO2_24g.xlsx
Wczytywanie: .\2018\2018_SO2_24g.xlsx
Wczytywanie: .\2019\2019_SO2_24g.xlsx
Wczytywanie: .\2020\2020_SO2_24g.xlsx
Wczytywanie: .\2021\2021_SO2_24g.xlsx
Wczytywanie: .\2022\2022_SO2_24g.xlsx
Wczytywanie: .\2023\2023_SO2_24g.xlsx
            DsWrocUkr  DsWrocWie  Kp

C:\Users\szymo\AppData\Local\Temp\ipykernel_948\1181867703.py:90: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)


✔️ Zapisano mapę: SO2.html

📊 Przetwarzam: PM25_24g
Wczytywanie: .\2002\2002_PM25_24g.xlsx
Wczytywanie: .\2003\2003_PM25_24g.xlsx
Wczytywanie: .\2004\2004_PM25_24g.xlsx
Wczytywanie: .\2005\2005_PM25_24g.xlsx
Wczytywanie: .\2006\2006_PM25_24g.xlsx
Wczytywanie: .\2007\2007_PM25_24g.xlsx
Wczytywanie: .\2008\2008_PM25_24g.xlsx
Wczytywanie: .\2009\2009_PM25_24g.xlsx
Wczytywanie: .\2010\2010_PM25_24g.xlsx
Wczytywanie: .\2011\2011_PM25_24g.xlsx
Wczytywanie: .\2012\2012_PM25_24g.xlsx
Wczytywanie: .\2013\2013_PM25_24g.xlsx
Wczytywanie: .\2014\2014_PM25_24g.xlsx
Wczytywanie: .\2015\2015_PM25_24g.xlsx
Wczytywanie: .\2016\2016_PM25_24g.xlsx
Wczytywanie: .\2017\2017_PM25_24g.xlsx
Wczytywanie: .\2018\2018_PM25_24g.xlsx
Wczytywanie: .\2019\2019_PM25_24g.xlsx
Wczytywanie: .\2020\2020_PM25_24g.xlsx
Wczytywanie: .\2021\2021_PM25_24g.xlsx
Wczytywanie: .\2022\2022_PM25_24g.xlsx
Wczytywanie: .\2023\2023_PM25_24g.xlsx
            MpKrakBulwar  MpKrakowWIOSPrad6115  LdLodzLegion  \
Data                      

C:\Users\szymo\AppData\Local\Temp\ipykernel_948\1181867703.py:90: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = df_renamed.groupby(axis=1, level=0).mean(numeric_only=True)


✔️ Zapisano mapę: PM25.html
